### 2.Import libraries and data

In [1]:
pip install wordcloud
pip install matplotlib_venn

SyntaxError: invalid syntax (<ipython-input-1-0519a017ecd4>, line 1)

In [ ]:
# maths
import scipy.stats as stats
import numpy as np
import pandas as pd
#from pandas.api.types import is_numeric_dtype

# visual
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
%matplotlib inline

# modelling
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

# nlp
import nltk
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
lemm = WordNetLemmatizer()

# web
from bs4 import BeautifulSoup
import requests
import json

# others
import time
import datetime as dt
import re
import os


### 3.Data cleaning & Exploratory Data Analysis

In [2]:
# clean data using re library and WordNetLemmatizer

wnl = WordNetLemmatizer()

def clean_text(text):
    
    #print(text)    
    text = text.lower() # To lowercase    
    text = re.sub(r'\&\w*;', '', text) # Remove HTML special entities (e.g. &amp;)        
    text = re.sub(r'\s\s+', ' ', text) # Remove whitespace (including new line characters)        
    text = re.sub(r'https?:\/\/.*\/\w*', '', text) # Remove hyperlinks 
    #print(text)
    
    words = text.split()
    #print(words)
    
    words_2 = []

    for word in words: 
        word_2 = wnl.lemmatize(word)
        #if word != word_2:
            #print(word,word_2)
        words_2.append(word_2)      

    output = ' '.join(words_2)
    #print(output)
    return output

NameError: name 'WordNetLemmatizer' is not defined

In [ ]:
#Read file & assign to pandas dataframes
xboxone = pd.read_csv("../data/xboxone.csv")

In [ ]:
#Read file & assign to pandas dataframes
ps4 = pd.read_csv("../data/ps4.csv")

In [ ]:
#check data shape
xboxone.shape, ps4.shape

In [ ]:
#check data head
xboxone.head()

In [ ]:
#check data head
ps4.head()

In [ ]:
# Dropping duplicates
xboxone = xboxone.drop_duplicates(["data"]).reset_index(drop=True)7

In [ ]:
# Dropping duplicates
ps4 = ps4.drop_duplicates(["data"]).reset_index(drop=True)

In [ ]:
#check for null values
xboxone.isnull().sum(), ps4.isnull().sum()

In [ ]:
#check data shape again
xboxone.shape, ps4.shape

In [ ]:
#Check data types
xboxone.dtypes, ps4.dtypes

In [ ]:
# Initialize the BeautifulSoup object on a single reddit   
example1 = BeautifulSoup(ps4['data'][0])

# Print the raw reddit and then the output of get_text(), for 
# comparison
print(xboxone['data'][1])
print()
print(example1.get_text())

In [ ]:
# Print the raw reddit and then the output of get_text(), for 
# comparison
print(ps4['data'][1])
print()
print(example1.get_text())

In [ ]:
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text())   # The text to search

In [ ]:
#Display columns
print(xboxone.columns, ps4.columns)

In [ ]:
#Display data types
xboxone.dtypes, ps4.dtypes

In [ ]:
#Display data summary
xboxone.describe(), ps4.describe()

In [ ]:
# Check for nulls in columns
null_cols = xboxone.isnull().sum()
mask_null = null_cols > 0
null_cols[mask_null].sort_values(ascending=False)

In [ ]:
# Check for nulls in rows
null_rows = xboxone.isnull().sum(axis=1)
mask_null = null_rows > 0
null_rows[mask_null].sort_values(ascending=False)

In [ ]:
# Check for nulls in columns
null_cols = ps4.isnull().sum()
mask_null = null_cols > 0
null_cols[mask_null].sort_values(ascending=False)

In [ ]:
# Check for nulls in rows
null_rows = ps4.isnull().sum(axis=1)
mask_null = null_rows > 0
null_rows[mask_null].sort_values(ascending=False)

In [ ]:
# show column summary
xboxone.info(),ps4.info()

In [ ]:
# clean text for xboxone and ps4
xboxone['data'] = xboxone['data'].apply(clean_text)
ps4['data'] = ps4['data'].apply(clean_text)

In [ ]:
# print top words in xboxone

cvec = CountVectorizer(stop_words='english')
cvec.fit(xboxone['data'])

df_xboxone = pd.DataFrame(cvec.transform(xboxone['data']).todense(),columns=cvec.get_feature_names())

top_xboxone = df_xboxone.sum(axis=0)
top_xboxone.sort_values(ascending = False).head(20)

In [ ]:
# print top words in ps4

cvec = CountVectorizer(stop_words='english')
cvec.fit(ps4['data'])

df_ps4 = pd.DataFrame(cvec.transform(ps4['data']).todense(),columns=cvec.get_feature_names())

top_ps4 = df_ps4.sum(axis=0)
top_ps4.sort_values(ascending = False).head(20)

In [ ]:
# get top words in xboxone

df_xboxone = top_xboxone.to_frame(name='count')
df_xboxone['word'] = df_xboxone.index
df_xboxone.reset_index(drop=True, inplace=True)
df_xboxone['freq'] = df_xboxone['count'] / len(xboxone) * 100

cols = ['word','count','freq']
df_xboxone = df_xboxone[cols]
df_xboxone.sort_values(by='count',ascending=False, inplace=True)
df_xboxone.head(10)

In [ ]:
# get top words in ps4

df_ps4 = top_ps4.to_frame(name='count')
df_ps4['word'] = df_ps4.index
df_ps4.reset_index(drop=True, inplace=True)
df_ps4['freq'] = df_ps4['count'] / len(ps4) * 100

cols = ['word','count','freq']
df_ps4 = df_ps4[cols]
df_ps4.sort_values(by='count',ascending=False, inplace=True)
df_ps4.head(10)

In [ ]:
plt.figure(figsize=(20,10))
plt.title('xboxone: Top 20 Words',fontsize=25)

sns.set_style("whitegrid")
sns.barplot(data=df_xboxone.head(20),x='freq',y='word',orient='h')

plt.xlabel('frequency (%)',fontsize=20)
plt.ylabel('word',fontsize=20)
plt.tick_params(labelsize=15)

In [ ]:
plt.figure(figsize=(20,10))
plt.title('ps4: Top 20 Words',fontsize=25)

sns.set_style("whitegrid")
sns.barplot(data=df_ps4.head(20),x='freq',y='word',orient='h')

plt.xlabel('frequency (%)',fontsize=20)
plt.ylabel('word',fontsize=20)
plt.tick_params(labelsize=15)

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Word Comparsion',fontsize=25)

set_a = set(df_xboxone['word'][:20])
set_b = set(df_ps4['word'][:20])

venn = venn2([set_a,set_b],set_labels=['xboxone','ps4'])

venn.get_label_by_id('100').set_text('\n'.join(map(str,set_a-set_b)))
venn.get_label_by_id('110').set_text('\n'.join(map(str,set_a&set_b)))
venn.get_label_by_id('010').set_text('\n'.join(map(str,set_b-set_a)))

venn.get_label_by_id('A').set_size(20)
venn.get_label_by_id('B').set_size(20)
venn.get_label_by_id('100').set_size(13)
venn.get_label_by_id('110').set_size(13)
venn.get_label_by_id('010').set_size(13)

In [ ]:
xboxone.isnull().sum(), ps4.isnull().sum()

In [ ]:
#Insert new column
xboxone['console'] = 1

In [ ]:
#Check header
xboxone.head()

In [ ]:
#Insert new column
ps4['console'] = 0

In [ ]:
#Check header
ps4.head()

In [ ]:
# create df by appending xboxone and ps4
df = xboxone.append(ps4, sort=False)

In [ ]:
#Check shape
df.shape

In [ ]:
#Drop null values
df.dropna(subset=['data'], inplace=True)

In [ ]:
#Check for null
df.isnull().sum()

In [ ]:
#Check header
df.head()

In [ ]:
def remove_redundant_words(string):
    df['data'] = df['data'].str.replace(string, '')

In [ ]:
redundant = ['xboxone',
             'xbox',
             'ps4',
             'false',
             'true',
             'all_ads',
             'approved_at_utc',
             'subreddit',
             'self',
             'text',
             'div',
             'game',
             '39',
             'thumbnail_height',
             'none',
             'flair'
            ]
for x in redundant:
  remove_redundant_words(x)

In [ ]:
def word_count(input_string):
    tokenizer = RegexpTokenizer(r'[A-Za-z0-9]{2,}')
    words = tokenizer.tokenize(input_string)
    counter.update(words)

In [ ]:
# Create a word frequency dictionary from the stemmed text for XBox One posts
counter = Counter()
for input_string in df[df['console']==1]['data']:
    word_count(input_string)
xboxone_word_freq = dict(counter.most_common())
# Generate a word cloud image
wordcloud = WordCloud(max_font_size=50).generate_from_frequencies(xboxone_word_freq)
plt.figure(figsize=(8,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud for Xbox One posts")
plt.show()

In [ ]:
# Create a word frequency dictionary from the stemmed text for PS4 posts
counter = Counter()
for input_string in df[df['console']==0]['data']:
    word_count(input_string)
ps4_word_freq = dict(counter.most_common())
# Generate a word cloud image
wordcloud = WordCloud(max_font_size=50).generate_from_frequencies(ps4_word_freq)
plt.figure(figsize=(8,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud for PS4 posts")
plt.show()

In [ ]:
# create X (feature matrix) and y (target)

X=df[['data']] 
y=df['console']

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,stratify=y,random_state=42)

In [ ]:
# Function to convert a raw string of text to a string of words
def text_processer(raw_text):

    #Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", raw_text)
    #Convert to lower case, split into individual words
    words = letters_only.lower().split()
    #Lemmatize words
    lemmed_words = [lemm.lemmatize(i) for i in words]
    #convert the stop words to a set
    stops = set(stopwords.words('english'))
    #Remove stop words
    meaningful_words = [w for w in lemmed_words if not w in stops]
    #Remove redundant words
    thing = ['xboxone']
    meaningful_words = [w for w in lemmed_words if not w in thing]    
    #Join words back into one string separated by space and return result
    return(" ".join(meaningful_words))

In [ ]:
# Get the number of documents based on the dataframe size.
data_train = X_train
total_train_data = len(data_train)
print(f'There are {total_train_data} train documents.')

data_test = X_test
total_test_data = len(data_test)
print(f'There are {total_test_data} test documents.')


# Initialize an empty list to hold the clean reviews.
clean_train_documents = []
clean_test_documents = []

In [ ]:
print("Cleaning and parsing the training set documents...")

j = 0
for train_documents in data_train['data']:
    # Convert review to words, then append to clean_train_reviews.
    clean_train_documents.append(text_processer(train_documents))
    
    # If the index is divisible by 200, print a message
    if (j + 1) % 200 == 0:
        print(f'Document {j + 1} of {total_train_data}.')
    
    j += 1

print("Cleaning and parsing the testing set documents...")

k = 0
for test_documents in data_test['data']:
    # Convert review to words, then append to clean_train_reviews.
    clean_test_documents.append(text_processer(test_documents))
    
    # If the index is divisible by 200, print a message
    if (k + 1) % 200 == 0:
        print(f'Review {k + 1} of {total_test_data}.')
        
    k += 1

In [ ]:
clean_train_documents

In [ ]:
clean_test_documents

In [ ]:
# check train and test data

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y.value_counts(normalize=True)

In [ ]:
print('baseline score:0.523305')

### 4.Modeling
### 4.1Logistic Regression Model

In [ ]:
#Check header
X_train.head()

In [ ]:
#Check header
y_train.head()

In [ ]:
#create baseline model with CountVectorizer
cvec = CountVectorizer()
cvec.fit(clean_train_documents)

In [ ]:
#cvec transform
X_train_cvec = cvec.transform(clean_train_documents)
X_test_cvec = cvec.transform(clean_test_documents)

In [ ]:
#assign df
df = pd.DataFrame(X_train_cvec.toarray(),columns=cvec.get_feature_names())

In [ ]:
y_train

In [ ]:
X_train_cvec

In [ ]:
#use with Logistic Regression

lr = LogisticRegression()
lr.fit(X_train_cvec, y_train)

lr_score_train = lr.score(X_train_cvec, y_train)
lr_score_test = lr.score(X_test_cvec, y_test)
print('lr_score_train:',lr_score_train)
print('lr_score_test:',lr_score_test)

In [ ]:
len(cvec.get_feature_names())

In [ ]:
#check length
len(lr.coef_)

In [ ]:
#display df
df = pd.DataFrame({'Feature': cvec.get_feature_names(), 'log_coef': lr.coef_[0]})
df['coef'] = np.exp(df['log_coef'])
df['abs_coef'] = abs(df['coef'])
df.sort_values('abs_coef', ascending=False).head(30)

### 4.2Naive Bayes model

In [ ]:
# Instantiate model
nb = MultinomialNB()

In [ ]:
# Fit naive bayes model
model = nb.fit(X_train_cvec, y_train)

In [ ]:
# Generate predictions
predictions = model.predict(X_test_cvec)

In [ ]:
# Score model on training set.
model.score(X_train_cvec, y_train)

In [ ]:
# Score model on testing set.
model.score(X_test_cvec, y_test)

In [ ]:
# Generate a confusion matrix.
confusion_matrix(y_test, predictions)

In [ ]:
#assign confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

In [ ]:
#print confusion matrix
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

In [3]:
# confusion matrix metrics
accuracy = (tp+tn)/(tp+fp+fn+tn)
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
precision = tp/(tp+fp)

print('Accuracy:',round(accuracy,3))
print('Sensitivity:',round(sensitivity,3))
print('Specificity:',round(specificity,3))
print('Precision:',round(precision,3))
print('')

NameError: name 'tp' is not defined

### 4.3Logistic Regression Model with Pipeline

In [ ]:
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

In [ ]:
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)],
    'lr__penalty':['l1','l2'],
    'lr__solver':['liblinear'],
    'lr__C':np.logspace(-3, 1, 5)
}
gs_cvec_lr = GridSearchCV(pipe, param_grid=pipe_params, cv=3, verbose=1, n_jobs=-1)
gs_cvec_lr.fit(clean_train_documents, y_train)
print(gs_cvec_lr.best_score_)
gs_cvec_lr.best_params_

In [ ]:
gs_cvec_lr.score(clean_train_documents,y_train)

In [ ]:
gs_cvec_lr.score(clean_test_documents,y_test)

### 4.4MultinomialNB Model with Pipeline

In [ ]:
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('mnb', MultinomialNB())
])

In [ ]:
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2), (1,3)],
    'mnb__alpha': np.linspace(0.5, 1.5, 3),
    'mnb__fit_prior': [True, False],  
}
gs_cvec_mnb = GridSearchCV(pipe, param_grid=pipe_params, cv=3, verbose=1, n_jobs=-1)
gs_cvec_mnb.fit(clean_train_documents, y_train)
print(gs_cvec_mnb.best_score_)
gs_cvec_mnb.best_params_

In [ ]:
gs_cvec_mnb.score(clean_train_documents,y_train)

In [ ]:
gs_cvec_mnb.score(clean_test_documents,y_test)

### 5. Conclusion and Recommendations

Baseline score:0.523305

Logistic Regression Model with Pipeline training score: 0.9900662251655629

Logistic Regression Model with Pipeline testing score: 0.9682539682539683

MultinomialNB Model with Pipeline training score: 0.9688741721854305

MultinomialNB Model with Pipeline testing score: 0.9629629629629629

Based on the best results, Logistic Regression testing with Pipeline scored the better than MultinomialNB. However, both the results are extremely close to each other. Moreover, they are also very close to 1, which might hint at overfitting. Therefore, more data is required to address the overfitting problem. Success will be evaluated based on how much improvement over the baseline score.

This project has successfully trained a classifier to differentiate which subreddit the post came from. Consequently, Microsoft can use the classifier to efficiently filter posts from other social media platform for further analysis.